<a href="https://colab.research.google.com/github/GheorgheBriana/Big-data-faculty/blob/main/Laborator5_A_GroupBy_aggregrate_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Importați clasa SparkSession
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [6]:
#Să se creeze o sesiune Spark, asociindu-i numele de aplicație "aggs"
spark1 = SparkSession.builder.appName("aggs").getOrCreate()

In [7]:
from google.colab import files
uploaded = files.upload()

Saving sales_info.csv to sales_info.csv


In [9]:
#Citiți fișierul sales_info.csv într-un DataFrame, știind că schema datelor va fi determinată automat și că datele au antet.
df = spark1.read.csv("sales_info.csv", header = True, inferSchema = True)

In [10]:
#Vizualizati datele
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [11]:
#Afisati schema
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [12]:
#Metoda groupby
df.groupby('Company')

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

In [16]:
#Media
df.groupby('Company').mean()

DataFrame[Company: string, avg(Sales): double]

In [17]:
#Afisare medie
df.groupby('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [19]:
#Max
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [20]:
#Min
df.groupBy('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [23]:
#Count
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [24]:
#Metoda agg(agregare)
#Sa se afiseze suma vanzarilor
df.agg({'Sales': 'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [25]:
#Sa se determine, in mod similar, valoarea maxima a vanzarilor
df.agg({'Sales': 'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [27]:
#Combinarea metodelor groupby si agg pentru a obtine informatii din companie
group_data = df.groupby('Company')

In [28]:
group_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [31]:
#Importul și utilizarea altor funcții
from pyspark.sql.functions import stddev, countDistinct, count, avg

In [32]:
df.agg({'Sales': 'stddev'}).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [33]:
#Rezultatul anterior poate fi obținut și prin utilizarea metodei `select`
df.select(stddev('Sales')).show()

+------------------+
|     stddev(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [34]:
#Selecteaza numarul de vanzari
df.select(count('Sales')).show()

+------------+
|count(Sales)|
+------------+
|          12|
+------------+



In [35]:
#Selecteaza numarul de vanzari distincte
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [37]:
#Formatarea rezultatelor
from pyspark.sql.functions import format_number

In [39]:
sales_std = df.select(stddev('Sales').alias('std'))

In [40]:
sales_std

DataFrame[std: double]

In [41]:
#Sa se formateze coloana std cu numar de 2 zecimale. Rezultatul va avea alias-ul "final"
sales_std = sales_std.select(format_number('std', 2).alias('final'))

In [42]:
sales_std

DataFrame[final: string]

In [43]:
sales_std.show()

+------+
| final|
+------+
|250.09|
+------+



In [44]:
#Utilizarea metodei orderBy pentru a ordona dupa valorile unei coloane
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [48]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [49]:
#Ordonare descrescator
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [ ]:
#Să se afle compania având cea mai mare valoare totală a vânzărilor.

In [50]:
#grupare date dupa companie
df.groupBy('Company').sum('Sales').show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [56]:
#Ordonare descrescatoare
df.groupBy('Company').sum('Sales').orderBy('sum(Sales)', ascending=False).show(1)

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
+-------+----------+
only showing top 1 row

